Lateral line detection in the video

In [300]:
import cv2
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas as pd


Load video

In [301]:
video_number = "7"
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = project_root / "data" / f"recording_{video_number}" / f"Recording_{video_number}.mp4" 
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Check
print(f"Opened: {cap.isOpened()}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, Total Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}")

Opened: True, FPS: 59.94018018162597, Total Frames: 332.0


Import horizontal lines

In [302]:
# Define the path to the CSV file
input_data_path = project_root / "data"/ "auxiliary_data" / "lane_lines" / f"horizontal_lines_{video_number}.csv"

# Load the CSV file into a DataFrame
horizontal_lines = pd.read_csv(input_data_path)


Import the average position of the ball

In [303]:
# Define the path to the CSV file
input_center_path = project_root / "data"/ "auxiliary_data" / "circle_position" / "METTINOMEGIUSTO.csv"

# Controlla se il file esiste
if Path(input_center_path).exists():
    # Carica il CSV in un DataFrame
    central_point = pd.read_csv(input_center_path)
else:
    # Se il file non esiste, imposta central_point a None
    central_point = None

Define Functions

In [304]:
# Funzione per estendere una linea
def extend_line(x1, y1, x2, y2, length=1000):
    # Calcola la lunghezza originale della linea
    dx, dy = x2 - x1, y2 - y1
    norm = np.sqrt(dx**2 + dy**2)  # Distanza euclidea tra i due punti

    # Evita divisioni per zero
    if norm == 0:
        return x1, y1, x2, y2  

    # Calcola i punti estesi
    x1_ext = int(x1 - length * (dx / norm))
    y1_ext = int(y1 - length * (dy / norm))
    x2_ext = int(x2 + length * (dx / norm))
    y2_ext = int(y2 + length * (dy / norm))

    return x1_ext, y1_ext, x2_ext, y2_ext

In [305]:
'''Disegna la linea sul frame'''
def write_line_on_frame(frame, line):
    # Create a copy of the original frame to draw the first line
    modified_frame = np.copy(frame)

    if line is not None:
        x1, y1, x2, y2 = line
        
       # Allunga la linea di 1000 pixel da entrambe le estremità
        x1_ext, y1_ext, x2_ext, y2_ext = extend_line(x1, y1, x2, y2, length=1000)
        
        # Disegna la linea estesa
        cv2.line(modified_frame, (x1_ext, y1_ext), (x2_ext, y2_ext), (0, 255, 0), 2)

    # return the modified frame
    return modified_frame

In [306]:
def write_lines_on_frame(frame, lines):
    for i in range(len(lines)):
        frame = write_line_on_frame(frame, lines[i])
    frame = write_line_on_frame(frame, [960, 0, 960, 1300])
    return frame

In [307]:
def polar_to_cartesian(line):
    rho, theta = line
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = x0 + 1000 * (-b)
    y1 = y0 + 1000 * (a)
    x2 = x0 - 1000 * (-b)
    y2 = y0 - 1000 * (a)
    return np.array([[x1, y1, x2, y2]])

In [308]:
def cartesian_to_polar(line):
    print(line)
    if line is not None:
        x1, y1, x2, y2 = line[0]
        # Compute rho and theta using the Hough Transform formula
        a = x2 - x1
        b = y2 - y1
        rho = abs(x1 * b - y1 * a) / math.sqrt(a**2 + b**2)
        theta = math.atan2(a, b)
        return rho, theta
    else:
        return None

In [309]:
def select_central_point(line, frame):
    rho, theta = line
    a = np.cos(theta)
    b = np.sin(theta)

    # Get the x-size of the image
    x_size = frame.shape[1]

    # Calculate the x-coordinate that is half of the x-size
    x_half = x_size // 2
    print('b:', b)
    # Calculate the corresponding y-coordinate using the line equation
    y_half = int((rho - x_half * a) / b)

    return x_half, y_half

In [310]:
def get_edges(frame):
    # Define the range for light brown color in HSV
    lower_brown = np.array([00, 30, 100])
    upper_brown = np.array([20, 200, 255])

    # Define the range for rose color in HSV
    lower_rose = np.array([150, 30, 200])
    upper_rose = np.array([180, 200, 255])

    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create masks for brown and rose colors
    mask_brown = cv2.inRange(hsv_image, lower_brown, upper_brown)
    mask_rose = cv2.inRange(hsv_image, lower_rose, upper_rose)

    # Combine the masks
    combined_mask = cv2.bitwise_or(mask_brown, mask_rose)

    # apply brown and rose mask
    extracted_image = cv2.bitwise_and(frame, frame, mask=combined_mask)

    # blur the image
    blurred_image = cv2.GaussianBlur(extracted_image, (15, 15), 0)

    # Convert the bottom image to grayscale
    gray_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)

    # Compute Otsu's threshold 
    otsu_thresh, _ = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Set lower and upper thresholds relative to Otsu's threshold
    lower = 0.5 * otsu_thresh
    upper = 1.5 * otsu_thresh

    # get edges
    edges = cv2.Canny(gray_image, lower, upper)

    return edges

In [311]:
def get_lines(edges):
    # Apply Probabilistic Hough Line Transform (allow to set minLineLength and maxLineGap)
    min_line_length = 50
    max_line_gap = 5 
    lines_p = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=min_line_length, maxLineGap=max_line_gap) 

    return lines_p

In [312]:
# Function to calculate the angle of a line
def calculate_angle(x1, y1, x2, y2):
    return math.degrees(math.atan2(y2 - y1, x2 - x1))

In [313]:
def cartesian_to_homogeneous(line):
    hom_line = np.cross([line[0], line[1], 1], [line[2], line[3], 1])
    hom_line = hom_line/hom_line[2]

    return hom_line

In [314]:
def compute_intersection(lines, horizontal_line):
    intersection_points_x = []
    # compute the intersection of each line with the horizontal line
    for line in lines:
        # x1, y1, x2, y2 = line[0]
        # x3, y3, x4, y4 = horizontal_line[0]

        # # Calculate the determinant
        # denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)

        # # Check if lines are parallel (denominator is zero)
        # if denominator == 0:
        #     continue

        # # Calculate the intersection point
        # px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator
        # py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator

        hom_line = cartesian_to_homogeneous(line[0])
        hom_hor_line = cartesian_to_homogeneous(horizontal_line[0])

        int_point = np.cross(hom_line, hom_hor_line)
        int_point = int_point / int_point[2]

        intersection_points_x.append(int_point[0])

    return intersection_points_x

In [315]:
def select_closest_lines(lines, horizontal_line, center):
    # compute the intersection of the lines with the horizontal lines
    intersections_points_x = compute_intersection(lines, horizontal_line)
    left_lines = []
    right_lines = []
    left_distances = []
    right_distances = []
    left_position = []
    for i in range(len(lines)):
        # if the intersection in at the left of the center
        if intersections_points_x[i] < center:
            left_lines.append(lines[i])
            left_distances.append(abs(center - intersections_points_x[i]))
            left_position.append(intersections_points_x[i])

        else: # if the intersection is at the right of the center
            right_lines.append(lines[i])
            right_distances.append(abs(center - intersections_points_x[i]))

    #compute the indeces of the minimum distance point
    min_left_index = left_distances.index(min(left_distances)) if left_distances else None
    min_right_index = right_distances.index(min(right_distances)) if right_distances else None

    # if exists, return the lines closest to the point
    if min_left_index is None:
        if min_right_index is None:
            return None, None
        return None, right_lines[min_right_index]
    
    if min_right_index is None:
        return left_lines[min_left_index], None

    return left_lines[min_left_index], right_lines[min_right_index]

In [316]:
def filter_lines(lines_p, horizontal_line, image_center, tolerance_angle = 20):
    # Filter the lines that have both endpoints over the horizontal line
    

    # Calculate the homogeneous coordinates of the horizontal line
    x1, y1, x2, y2 = horizontal_line[0]
    horizontal_line_homogeneous = np.cross([x1, y1, 1], [x2, y2, 1])
    horizontal_line_homogeneous = horizontal_line_homogeneous / horizontal_line_homogeneous[0]

    # Filter out lines that are 'quite horizontal' with a tolerance of 20 degrees
    # and are in the bottom quarter of the image
    filtered_lines = []
    if lines_p is not None:
        for line in lines_p:
            x1, y1, x2, y2 = line[0]
            angle = calculate_angle(x1, y1, x2, y2)
            if abs(angle) > tolerance_angle:
                y_max = max(y1, y2)
                x_max = x1 if y_max == y1 else x2
                if x_max + y_max * horizontal_line_homogeneous[1] + horizontal_line_homogeneous[2] > 0: # se a*x + b*y + c > 0 allora il punto è sopra la linea
                    filtered_lines.append(line)

    if len(filtered_lines) == 0:
        return None, None

    # divide the lines in left and right and select the closests
    left_line, right_line = select_closest_lines(filtered_lines, horizontal_line, image_center)
 
    return left_line, right_line

In [317]:
# # Function to calculate the distance from a point to a line
# def distance_point_to_line(x0, y0, x1, y1, x2, y2):
#     return abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1) / math.sqrt((y2 - y1)**2 + (x2 - x1)**2)


In [318]:
# def find_closest_line(lines, central_point):
#     # Initialize the minimum distance and the closest line
#     min_distance = float('inf')
#     closest_line = None
    
#     # Iterate through the filtered lines to find the closest one
#     for line in lines:
#         x1, y1, x2, y2 = line[0]
#         distance = distance_point_to_line(central_point[0], central_point[1], x1, y1, x2, y2)
#         if distance < min_distance:
#             min_distance = distance
#             closest_line = line

#     return closest_line

In [319]:
# '''rivedi bene che linee ti lascia prima che faccia casini, in teoria non funziona bene'''
# def filter_similar_lines(lines, reference_line, tolerance_angle=30, equality_treshold=2):
#     if len(lines) > 0:
#         # Calculate the angle of the closest line
#         x1_closest, y1_closest, x2_closest, y2_closest = reference_line[0]
#         angle_closest = calculate_angle(x1_closest, y1_closest, x2_closest, y2_closest)
#         print('dimensione dell array prima di eliminare la linea:', len(lines))
#         # Remove the reference line from the list of lines
#         lines = [line for line in lines if not np.array_equal(line, reference_line)]
#         print('dimensione dell array dopo aver eliminato la linea:', len(lines))
#         # Filter lines based on angle similarity with a tolerance
#         similar_angle_lines = []
#         for line in lines:
#             x1, y1, x2, y2 = line[0]
#             angle = calculate_angle(x1, y1, x2, y2)
#             if abs(angle - angle_closest) > equality_treshold: #<= tolerance_angle:
#                 similar_angle_lines.append(line)
#         if len(similar_angle_lines) > 0:
#             return similar_angle_lines
#     return None

In [320]:
def compute_lines(frame, horizontal_line, central_point):
    null_line = np.array([[0, 0, 0, 0]])
    all_lines = []
    horizontal_line_carteisan = polar_to_cartesian(horizontal_line)
    # add the horizontal line
    all_lines.append(horizontal_line_carteisan[0])
    # define the central point
    if not central_point:
        central_point = select_central_point(horizontal_line, frame)
    # compute the edges
    edges = get_edges(frame)
    # get the lines
    lines_p = get_lines(edges)

    # select the closest left an right line
    left_line, right_line = filter_lines(lines_p, horizontal_line_carteisan, central_point[0]) 

    # add the lines to the list
    if left_line is None:
        all_lines.append(null_line[0])
    else:
        all_lines.append(left_line[0])

    if right_line is None:
        all_lines.append(null_line[0])
    else:
        all_lines.append(right_line[0])

    print('left:', left_line, 'right:', right_line)

    # if lines_filtered is not None:
    #     closest_line = find_closest_line(lines_filtered, central_point)
    #     all_lines.append(closest_line[0])
    #     similar_lines = filter_similar_lines(lines_filtered, closest_line)
    #     if similar_lines is not None:
    #         second_closest_line = find_closest_line(similar_lines, central_point)
    #         all_lines.append(second_closest_line[0])
    #     else:
    #         all_lines.append(null_line[0])
    # else:
    #     all_lines.append(null_line[0])
    #     all_lines.append(null_line[0])
        
    return all_lines
 

Generate Video

In [321]:
# Reset the video to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Define the codec and create a VideoWriter object to save the modified frames
output_path = project_root / "data" / f"recording_{video_number}" / "Lines_video_non_processed.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec for MP4 format
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))

# Loop through each frame in the video
frame_index = 0
lines_array = []
print(len(horizontal_lines))
while frame_index < len(horizontal_lines):
    ret, video_frame = cap.read()
    if not ret:
        print("End of video or failed to read the frame at iteration", frame_index)
        break

    # Compute the three lines in the frame
    all_lines = compute_lines(video_frame, horizontal_lines.iloc[frame_index], central_point) 

    if all_lines is None:
        print("No lines found in frame", frame_index)
        modified_frame = video_frame
        
    else:
        # draw the lines on the frame   
        modified_frame = write_lines_on_frame(video_frame, all_lines)
        # add lines to the output array
        lines_array.append(all_lines)

    # Write the modified frame to the output video
    out.write(modified_frame)

    # Increment the frame index
    frame_index += 1

# Release the video capture and writer objects
# cap.release()
out.release()

print(f"Adjusted video saved to {output_path}")

332
b: 0.9989685402102996
left: [[1200  316 1253  263]] right: [[306 131 381 217]]
b: 0.9989685402102996
left: [[1060  456 1121  393]] right: [[1278 1035 1301  939]]
b: 0.9989685402102996
left: [[ 996  521 1058  459]] right: [[1316  874 1329  820]]
b: 0.9989685402102996
left: [[ 996  521 1058  459]] right: [[1303  926 1329  821]]
b: 0.9989685402102996
left: [[1182  335 1238  277]] right: [[1303  926 1326  833]]
b: 0.9989685402102996
left: [[ 996  521 1058  459]] right: [[1286 1004 1301  937]]
b: 0.9989685402102996
left: [[ 996  521 1058  459]] right: [[1304  925 1318  868]]
b: 0.9989685402102996
left: [[ 996  521 1058  459]] right: [[1285 1006 1301  936]]
b: 0.9989685402102996
left: [[ 996  521 1058  459]] right: [[1304  925 1321  854]]
b: 0.9989685402102996
left: [[ 996  521 1058  459]] right: [[1304  925 1323  844]]
b: 0.9989685402102996
left: [[ 996  521 1058  459]] right: [[1278 1035 1301  939]]
b: 0.9989685402102996
left: [[ 511 1008  577  942]] right: [[1285 1006 1298  953]]
b: 0

Save lines in csv file

In [322]:
# Define the output path for the lines CSV file
output_lines_path = project_root / "data" / "auxiliary_data" / "lane_lines" / f"three_lines_{video_number}.csv"

# Convert lines_array to a DataFrame
lines_df = pd.DataFrame(lines_array)

# Save the DataFrame to a CSV file
lines_df.to_csv(output_lines_path, index=False)

print(f"Lines array saved to {output_lines_path}")

Lines array saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\data\auxiliary_data\lane_lines\three_lines_7.csv
